In [1]:
import os
os.chdir('../')

In [2]:
from rsna_retro.imports import *
from rsna_retro.metadata import *
from rsna_retro.preprocess import *
from rsna_retro.train import *

Loading imports


In [3]:
torch.cuda.set_device(1)

In [4]:
Meta.df_comb.head()

,Modality,PatientID,StudyInstanceUID,SeriesInstanceUID,StudyID,ImagePositionPatient,ImageOrientationPatient,SamplesPerPixel,PhotometricInterpretation,Rows,...,MultiWindowCenter,WindowCenter1,MultiWindowWidth,WindowWidth1,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural
SOPInstanceUID,,,,,,,,,,,,,,,,,,,,,
ID_76d55d9d0,CT,ID_b9797064,ID_00b9e1961f,ID_0000298a7d,,-125.0,1.0,1,MONOCHROME2,512,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
ID_96d282ea9,CT,ID_b9797064,ID_00b9e1961f,ID_0000298a7d,,-125.0,1.0,1,MONOCHROME2,512,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
ID_7d8a7c29d,CT,ID_b9797064,ID_00b9e1961f,ID_0000298a7d,,-125.0,1.0,1,MONOCHROME2,512,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
ID_4d4401491,CT,ID_b9797064,ID_00b9e1961f,ID_0000298a7d,,-125.0,1.0,1,MONOCHROME2,512,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
ID_8f5ded0b7,CT,ID_b9797064,ID_00b9e1961f,ID_0000298a7d,,-125.0,1.0,1,MONOCHROME2,512,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0


## Training

In [5]:
name = 'baseline_full_stg1_xresnet18'

In [6]:
dbch = get_data(512, 128, splits=Meta.splits_stg1)
learn = get_learner(dbch, xse_resnext18, pretrained=True)

## Training

In [7]:
do_fit(learn, 12, 4e-2)
learn.save(f'runs/{name}-1')

epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time
0,0.123054,0.148408,0.959785,0.912432,08:04
1,0.108669,0.116229,0.965455,0.927863,07:49
2,0.103368,0.131626,0.966457,0.930791,07:54
3,0.100138,0.112510,0.966346,0.933427,07:57
4,0.095349,0.103919,0.969050,0.937921,07:53
5,0.093843,0.107704,0.968865,0.935298,07:52
6,0.088968,0.100605,0.969612,0.934878,07:54
7,0.085113,0.096548,0.971853,0.942110,07:56
8,0.079056,0.094966,0.971683,0.942950,07:57
9,0.075653,0.090371,0.972825,0.944872,07:57


In [7]:
learn.load(f'runs/{name}-1')
learn.dbunch = get_data(512, 256, splits=Meta.splits_stg1)
do_fit(learn, 12, 4e-3)
learn.save(f'runs/{name}-2')

epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time
0,0.079286,0.092898,0.972385,0.945280,19:07
1,0.075121,0.092739,0.973173,0.946082,19:04
2,0.074387,0.089483,0.973448,0.946833,19:05
3,0.071403,0.087614,0.973822,0.947126,19:03
4,0.071591,0.084352,0.974356,0.949519,19:04
5,0.068402,0.085397,0.974963,0.950232,19:06
6,0.065008,0.084122,0.974885,0.951505,19:03
7,0.062272,0.083037,0.975638,0.952257,19:08
8,0.061048,0.083151,0.975723,0.952155,19:07
9,0.055590,0.085225,0.975891,0.952091,19:07


In [8]:
learn.dbunch = get_data(256, 384, splits=Meta.splits_stg1, img_dir=path_jpg)
do_fit(learn, 4, 4e-4)
learn.save(f'runs/{name}-3')

epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time
0,0.064307,0.081396,0.975570,0.952524,46:23
1,0.061411,0.080464,0.975804,0.953352,46:23
2,0.056917,0.081589,0.976041,0.953784,46:35
3,0.053236,0.081172,0.976120,0.954001,46:45


## Submission

In [7]:
learn.load(f'runs/{name}-3')
sub_fn = f'subm/{name}-3'

In [8]:
learn.dbunch = get_test_data(Meta.df_tst, bs=512, sz=384)

In [9]:
preds,targs = learn.get_preds()

In [10]:
pred_csv = submission(Meta.df_tst, preds, fn=sub_fn)

In [11]:
FileLink(f'{sub_fn}.csv')

/home/ashaw/kaggle/rsna_retro/subm/baseline_full_stg1_xresnet18-3.csv

In [12]:
api.competition_submit(f'{sub_fn}.csv', 'Training on Stage 1 (384), Valid on Stage 2. - 0.081172', 'rsna-intracranial-hemorrhage-detection')

100%|██████████| 26.0M/26.0M [00:02<00:00, 9.76MB/s]


Successfully submitted to RSNA Intracranial Hemorrhage Detection

In [14]:
api.competitions_submissions_list('rsna-intracranial-hemorrhage-detection')[0]

{'ref': 14276411,
 'totalBytes': 27277209,
 'date': '2020-01-23T19:25:44.513Z',
 'description': 'Training on Stage 1 (384), Valid on Stage 2. - 0.081172',
 'errorDescription': None,
 'fileName': 'baseline_full_stg1_xresnet18-3.csv',
 'publicScore': '0.53894',
 'privateScore': '0.06565',
 'status': 'complete',
 'submittedBy': 'Andrew Shaw',
 'submittedByRef': 'bearpelican',
 'teamName': 'Andrew Shaw',
 'type': 'standard',
 'url': 'https://www.kaggle.com/submissions/14276411/14276411.raw'}

In [25]:
api.competitions_submissions_list('rsna-intracranial-hemorrhage-detection')[0]

{'ref': 14258280,
 'totalBytes': 27277209,
 'date': '2020-01-22T06:54:06.2Z',
 'description': 'Training on Stage 1 (384), Valid on Stage 2. - 0.084161',
 'errorDescription': None,
 'fileName': 'baseline_full_stg1-3.csv',
 'publicScore': '0.61200',
 'privateScore': '0.06511',
 'status': 'complete',
 'submittedBy': 'Andrew Shaw',
 'submittedByRef': 'bearpelican',
 'teamName': 'Andrew Shaw',
 'type': 'standard',
 'url': 'https://www.kaggle.com/submissions/14258280/14258280.raw'}